In [2]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
import pandas as pd
from selenium import webdriver

from tqdm import tqdm
import os
from bs4 import BeautifulSoup
import requests
import numpy as np
import re


In [3]:
url_cats = f"https://www.maminess.com/categoria/ropa-premama/"
res_cats = requests.get(url_cats)
sopa_cats = BeautifulSoup(res_cats.content, "html.parser")
lista_cats = sopa_cats.findAll("ul", {"class" :"children"})

#<li class="cat-item cat-item-15 cat-item-ropa-premama root-item current-cat active cat-parent li_accordion"><a href="javascript:void(0);" class="accordion" data-class_show="pe-7s-plus" data-class_hide="pe-7s-less"><span class="icon pe-7s-less"></span></a><a href="https://www.maminess.com/categoria/ropa-premama/" data-id="15" class="nasa-filter-by-cat nasa-active">Ropa Premamá</a><ul class="children"><li class="cat-item cat-item-232 cat-item-vestidos-monos-premama"><a href="https://www.maminess.com/categoria/ropa-premama/vestidos-monos-premama/" data-id="232" class="nasa-filter-by-cat">Vestidos y monos Premamá</a></li><li class="cat-item cat-item-233 cat-item-tops-premama"><a href="https://www.maminess.com/categoria/ropa-premama/tops-premama/" data-id="233" class="nasa-filter-by-cat">Tops premamá</a></li><li class="cat-item cat-item-234 cat-item-pantalones-premama"><a href="https://www.maminess.com/categoria/ropa-premama/pantalones-premama/" data-id="234" class="nasa-filter-by-cat">Pantalones y Faldas</a></li><li class="cat-item cat-item-236 cat-item-banadores"><a href="https://www.maminess.com/categoria/ropa-premama/banadores/" data-id="236" class="nasa-filter-by-cat">Bañadores</a></li><li class="cat-item cat-item-237 cat-item-vestidos-fiesta-premama"><a href="https://www.maminess.com/categoria/ropa-premama/vestidos-fiesta-premama/" data-id="237" class="nasa-filter-by-cat">Vestidos de Fiesta</a></li><li class="cat-item cat-item-291 cat-item-pijamas-y-camisones"><a href="https://www.maminess.com/categoria/ropa-premama/pijamas-y-camisones/" data-id="291" class="nasa-filter-by-cat">Pijamas y Camisones</a></li><li class="cat-item cat-item-306 cat-item-promocion-hasta-60"><a href="https://www.maminess.com/categoria/ropa-premama/promocion-hasta-60/" data-id="306" class="nasa-filter-by-cat">Promo hasta -60%</a></li></ul></li>

In [6]:
lista_cats[0].getText()

'Vestidos y monos PremamáTops premamáPantalones y FaldasBañadoresVestidos de FiestaPijamas y CamisonesPromo hasta -60%'

In [ ]:
for cat in lista_cats:
    driver = webdriver.Chrome()
    url_mercados = f"https://www.maminess.com/categoria/ropa-premama/{cat}"
    driver.get(url_mercados)
    sleep(3)
    #aceptar cookie
    driver.find_element("css selector", "#rcc-confirm-button").click()

    for pag in range(1,3):
        url_super = f"https://www.maminess.com/categoria/ropa-premama/pantalones-premama/page/{pag}/"
        res_facua = requests.get(url_super)
        sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
        lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})

In [ ]:
driver = webdriver.Chrome()
#aquí habría un for de pagina
url_mercados = f"https://www.maminess.com/categoria/ropa-premama/"
driver.get(url_mercados)
driver.maximize_window()
sleep(1)

#aceptar cookie
driver.find_element("css selector", "#cmplz-cookiebanner-container > div > div.cmplz-buttons > button.cmplz-btn.cmplz-accept").click()
sleep(2)

#pinchar en la categoria vestidos
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a").click()
sleep(8)


# element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a")))
# element.click()

#pinchar en cada prod para sacar info
lista_nombres=[]
lista_precios=[]
lista_nvaloraciones=[]
try:
    for nfila in range(1,5):#si no hay tantas, que salga del bucle
        try:
            for ncol in range(1,5):

                sleep(2)
                driver.find_element("css selector", f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a").click()
                                                      
                sleep(2)

                #Beauti
                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")




                #Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class" :"product_title entry-title"})
                nombre=lista_nom[0].getText().lower().strip()
                lista_nombres.append(nombre)

                #Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class" :"woocommerce-Price-amount amount"})
                precio=lista_prec[1].getText().replace(",",".").replace("€", "")
                lista_precios.append(precio)


                #Conseguir valoraciones(Creo que no tiene)h5
                lista_val = sopa_producto.findAll("div", {"class" :"nasa-tab-wrap text-center"})

                texto=lista_val[0].getText()
                numero= re.search(r'\((\d+)\)', texto)
                nvaloraciones= numero.group(1)
                lista_nvaloraciones.append(nvaloraciones)
                driver.back()
        except:
            driver.execute_script("window.scrollBy(0, 300);")
            continue
except:
    pass #habrá que cambiarlo a que vuelva al for de pag si se acaban las filas



#pinchar en la categoria pantalones/leggins



#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-234.cat-item-pantalones-premama > a

vamos a desarrollar el de pantalones

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver

driver = webdriver.Chrome()
# Cargar la página
url_mercados = f"https://www.maminess.com/categoria/ropa-premama/"
driver.get(url_mercados)
driver.maximize_window()
sleep(1)

# Aceptar cookies
driver.find_element("css selector", "#cmplz-cookiebanner-container > div > div.cmplz-buttons > button.cmplz-btn.cmplz-accept").click()
sleep(2)

# Pinchar en la categoría de vestidos
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a").click()
sleep(5)

# Listas para almacenar la información
lista_nombres = []
lista_precios = []
lista_nvaloraciones = []

# Bucle para recorrer filas y columnas de productos

for nfila in range(1, 5):  # Recorrer filas
    try:
        for ncol in range(1, 6):  # Recorrer columnas
            try:
                # Espera explícita para que el producto esté listo para hacer clic
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a"))
                ).click()
                
                # Espera tras hacer clic
                sleep(2)

                # Extraer información del producto

                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")
                
                # Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                lista_nombres.append(nombre)

                # Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                lista_precios.append(precio)
                
                # Conseguir valoraciones
                lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                if lista_val:
                    texto = lista_val[0].getText()
                    numero = re.search(r'\((\d+)\)', texto)
                    nvaloraciones = numero.group(1) if numero else "0"
                else:
                    nvaloraciones = "No hay valoraciones"
                lista_nvaloraciones.append(nvaloraciones)

                # Volver a la página de productos
                driver.back()
                
                # Espera explícita después de regresar a la página anterior
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#main .nasa-products-page-wrap"))
                )

            except Exception as e:
                print(f"Error al procesar el producto en fila {nfila} columna {ncol}: {e}")
                driver.execute_script("window.scrollBy(0, 300);")
                #aqui cambio pag
                if len(lista_nombres) == 13:
                    sleep(2)
                    driver.find_element("css selector", "#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.row.nasa-paginations-warp.filters-container-down > div > div > div > ul > li:nth-child(4) > a > span").click()
                    sleep(5)
                    
                    #l
                else:
                    continue
            
            
    except Exception as e:
        print(f"Error general: {e}")




# Cerrar el navegador


# Resultado
print(lista_nombres)
print(lista_precios)
print(lista_nvaloraciones)


KeyboardInterrupt: 

In [43]:
lista_precios

['34.90', '44.90', '54.90', '39.90']

In [36]:

lista_nombres

['jersey premamá punto chocolate leslie']

Este esta bien

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver

driver = webdriver.Chrome()
# Cargar la página
url_mercados = f"https://www.maminess.com/categoria/ropa-premama/"
driver.get(url_mercados)
driver.maximize_window()
sleep(1)

# Aceptar cookies
driver.find_element("css selector", "#cmplz-cookiebanner-container > div > div.cmplz-buttons > button.cmplz-btn.cmplz-accept").click()
sleep(2)

# Pinchar en la categoría de vestidos
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a").click()
sleep(5)

# Listas para almacenar la información
lista_nombres = []
lista_precios = []
lista_nvaloraciones = []

# Bucle para recorrer filas y columnas de productos
try:
    for nfila in range(1, 5):  # Recorrer filas
        for ncol in range(1, 5):  # Recorrer columnas
            try:
                # Espera explícita para que el producto esté listo para hacer clic
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a"))
                ).click()
                
                # Espera tras hacer clic
                sleep(2)

                # Extraer información del producto
                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")
                
                # Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                lista_nombres.append(nombre)

                # Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                lista_precios.append(precio)
                
                # Conseguir valoraciones
                lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                if lista_val:
                    texto = lista_val[0].getText()
                    numero = re.search(r'\((\d+)\)', texto)
                    nvaloraciones = numero.group(1) if numero else "0"
                else:
                    nvaloraciones = "No hay valoraciones"
                lista_nvaloraciones.append(nvaloraciones)

                # Volver a la página de productos
                driver.back()
                
                # Espera explícita después de regresar a la página anterior
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#main .nasa-products-page-wrap"))
                )

            except Exception as e:
                print(f"Error al procesar el producto en fila {nfila} columna {ncol}: {e}")
                driver.execute_script("window.scrollBy(0, 300);")
                continue

except Exception as e:
    print(f"Error general: {e}")

# Cerrar el navegador
driver.quit()

# Resultado
print(lista_nombres)
print(lista_precios)
print(lista_nvaloraciones)


este pruebo coger ambas cats

In [62]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver

driver = webdriver.Chrome()
# Cargar la página
url_mercados = f"https://www.maminess.com/categoria/ropa-premama/"
driver.get(url_mercados)
driver.maximize_window()
sleep(1)

# Aceptar cookies
driver.find_element("css selector", "#cmplz-cookiebanner-container > div > div.cmplz-buttons > button.cmplz-btn.cmplz-accept").click()
sleep(2)

# Pinchar en la categoría de vestidos
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a").click()
sleep(5)

# Listas para almacenar la información
lista_nombres = []
lista_precios = []
lista_nvaloraciones = []

# Bucle para recorrer filas y columnas de productos
try:
    for nfila in range(1, 5):  # Recorrer filas
        for ncol in range(1, 5):  # Recorrer columnas
            try:
                # Espera explícita para que el producto esté listo para hacer clic
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a"))
                ).click()
                
                # Espera tras hacer clic
                sleep(2)

                # Extraer información del producto
                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")
                
                # Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                lista_nombres.append(nombre)

                # Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                lista_precios.append(precio)
                
                # Conseguir valoraciones
                lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                if lista_val:
                    texto = lista_val[0].getText()
                    numero = re.search(r'\((\d+)\)', texto)
                    nvaloraciones = numero.group(1) if numero else "0"
                else:
                    nvaloraciones = "No hay valoraciones"
                lista_nvaloraciones.append(nvaloraciones)

                # Volver a la página de productos
                driver.back()
                
                # Espera explícita después de regresar a la página anterior
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#main .nasa-products-page-wrap"))
                )

            except Exception as e:
                print(f"Error al procesar el producto en fila {nfila} columna {ncol}: {e}")
                driver.execute_script("window.scrollBy(0, 300);")
                continue

except Exception as e:
    print(f"Error general: {e}")

#pinchar en la categoria pantalones/leggins
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-234.cat-item-pantalones-premama > a").click()
sleep(8)

lista_nombresp = []
lista_preciosp = []
lista_nvaloracionesp = []

# Bucle para recorrer filas y columnas de productos
try:
    for nfila in range(1, 5):  # Recorrer filas
        for ncol in range(1, 5):  # Recorrer columnas
            try:
                # Espera explícita para que el producto esté listo para hacer clic
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a"))
                ).click()
                
                # Espera tras hacer clic
                sleep(2)

                # Extraer información del producto
                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")
                
                # Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                lista_nombresp.append(nombre)

                # Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                lista_preciosp.append(precio)
                
                # Conseguir valoraciones
                lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                if lista_val:
                    texto = lista_val[0].getText()
                    numero = re.search(r'\((\d+)\)', texto)
                    nvaloraciones = numero.group(1) if numero else "0"
                else:
                    nvaloraciones = "No hay valoraciones"
                lista_nvaloracionesp.append(nvaloraciones)

                # Volver a la página de productos
                driver.back()
                
                # Espera explícita después de regresar a la página anterior
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#main .nasa-products-page-wrap"))
                )

            except Exception as e:
                print(f"Error al procesar el producto en fila {nfila} columna {ncol}: {e}")
                driver.execute_script("window.scrollBy(0, 300);")
                continue

except Exception as e:
    print(f"Error general: {e}")

# Cerrar el navegador
driver.quit()

# Resultado
print(lista_nombres)
print(lista_precios)
print(lista_nvaloraciones)
print(lista_nombresp)
print(lista_preciosp)
print(lista_nvaloracionesp)


Error al procesar el producto en fila 2 columna 1: Message: 

Error al procesar el producto en fila 3 columna 1: Message: 

Error al procesar el producto en fila 4 columna 1: Message: 



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-234.cat-item-pantalones-premama > a"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DD8A38A5+3004357]
	(No symbol) [0x00007FF6DD539970]
	(No symbol) [0x00007FF6DD3E582A]
	(No symbol) [0x00007FF6DD435B8E]
	(No symbol) [0x00007FF6DD435E7C]
	(No symbol) [0x00007FF6DD47EC27]
	(No symbol) [0x00007FF6DD45BC1F]
	(No symbol) [0x00007FF6DD47BA4C]
	(No symbol) [0x00007FF6DD45B983]
	(No symbol) [0x00007FF6DD427628]
	(No symbol) [0x00007FF6DD428791]
	GetHandleVerifier [0x00007FF6DD8CA00D+3161901]
	GetHandleVerifier [0x00007FF6DD91E060+3506048]
	GetHa

click en columnas2,3,4

In [59]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
import re

driver = webdriver.Chrome()
# Cargar la página
url_mercados = f"https://www.maminess.com/categoria/ropa-premama/"
driver.get(url_mercados)
driver.maximize_window()
sleep(1)

# Aceptar cookies
driver.find_element("css selector", "#cmplz-cookiebanner-container > div > div.cmplz-buttons > button.cmplz-btn.cmplz-accept").click()
sleep(2)

# Pinchar en la categoría de vestidos
driver.find_element("css selector", "#nasa_product_categories-6 > ul > li.cat-item.cat-item-15.cat-item-ropa-premama.root-item.current-cat.active.cat-parent.li_accordion > ul > li.cat-item.cat-item-232.cat-item-vestidos-monos-premama > a").click()
sleep(5)

# Listas para almacenar la información
lista_nombres = []
lista_precios = []
lista_nvaloraciones = []

# Bucle para recorrer filas y columnas de productos
try:
    for nfila in range(1, 5):  # Recorrer filas
        for ncol in range(1, 5):  # Recorrer columnas
            try:
                sleep(8)
                # Espera explícita para que el producto esté listo para hacer clic
                driver.find_element("css selector", f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila}) > div:nth-child({ncol}) > div > div > div > div > div.product-info-wrap > div > div.name > a").click()
                # Espera tras hacer clic
                sleep(2)

                # Extraer información del producto
                sopa_producto = BeautifulSoup(driver.page_source, "html.parser")
                
                # Conseguir nombre
                lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                lista_nombres.append(nombre)

                # Conseguir precio
                lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                lista_precios.append(precio)
                
                # Conseguir valoraciones
                lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                if lista_val:
                    texto = lista_val[0].getText()
                    numero = re.search(r'\((\d+)\)', texto)
                    nvaloraciones = numero.group(1) if numero else "0"
                else:
                    nvaloraciones = "No hay valoraciones"
                lista_nvaloraciones.append(nvaloraciones)

                # Volver a la página de productos
                driver.back()
                
                # Espera explícita después de regresar a la página anterior
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp > div > div > div > div:nth-child({nfila})"))
                )
                
                # Espera adicional tras la recarga
                sleep(1)

            except Exception as e:
                print(f"Error al procesar el producto en fila {nfila} columna {ncol}: {e}")
                driver.execute_script("window.scrollBy(0, 300);")
                continue

except Exception as e:
    print(f"Error general: {e}")

# Cerrar el navegador
driver.quit()

# Resultado
print(lista_nombres)
print(lista_precios)
print(lista_nvaloraciones)


Error al procesar el producto en fila 2 columna 1: Message: element not interactable
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF6DD8A38A5+3004357]
	(No symbol) [0x00007FF6DD539970]
	(No symbol) [0x00007FF6DD3E5659]
	(No symbol) [0x00007FF6DD43702F]
	(No symbol) [0x00007FF6DD42996E]
	(No symbol) [0x00007FF6DD45BBDA]
	(No symbol) [0x00007FF6DD4292A6]
	(No symbol) [0x00007FF6DD45BDF0]
	(No symbol) [0x00007FF6DD47BA4C]
	(No symbol) [0x00007FF6DD45B983]
	(No symbol) [0x00007FF6DD427628]
	(No symbol) [0x00007FF6DD428791]
	GetHandleVerifier [0x00007FF6DD8CA00D+3161901]
	GetHandleVerifier [0x00007FF6DD91E060+3506048]
	GetHandleVerifier [0x00007FF6DD91400D+3465005]
	GetHandleVerifier [0x00007FF6DD690EEB+830987]
	(No symbol) [0x00007FF6DD54467F]
	(No symbol) [0x00007FF6DD5409D4]
	(No symbol) [0x00007FF6DD540B6D]
	(No symbol) [0x00007FF6DD530149]
	BaseThreadInitThunk [0x00007FF989A6257D+29]
	RtlUserThreadStart [0x00007FF98BB4AF08+40]



KeyboardInterrupt: 

NUEVO EMPIECE

Como no tiene valoraciones voy a hacer que de cada pagina me coja su nombre y precio solo

In [ ]:
lista_url_cat=["vestidos-monos-premama/", "ropa-premama/pantalones-premama/"]
for cat in lista_url_cat:
    for pag in range(1,4):
        try:
            url_mercados = f"https://www.maminess.com/categoria/ropa-premama/{cat}page/{pag}/"
            res_producto = requests.get(url_mercados)
            sopa_producto = BeautifulSoup(res_producto.content, "html.parser")
            
                        # Conseguir nombre
                        lista_nom = sopa_producto.findAll("h1", {"class": "product_title entry-title"})
                        nombre = lista_nom[0].getText().lower().strip() if lista_nom else "Nombre no encontrado"
                        lista_nombres.append(nombre)

                        # Conseguir precio
                        lista_prec = sopa_producto.findAll("span", {"class": "woocommerce-Price-amount amount"})
                        precio = lista_prec[1].getText().replace(",", ".").replace("€", "") if len(lista_prec) > 1 else "Precio no encontrado"
                        lista_precios.append(precio)
                        
                        # Conseguir valoraciones
                        lista_val = sopa_producto.findAll("div", {"class": "nasa-tab-wrap text-center"})
                        if lista_val:
                            texto = lista_val[0].getText()
                            numero = re.search(r'\((\d+)\)', texto)
                            nvaloraciones = numero.group(1) if numero else "0"
                        else:
                            nvaloraciones = "No hay valoraciones"
                        lista_nvaloraciones.append(nvaloraciones)

                        # Volver a la página de productos
                        driver.back()
                        
                        # Espera explícita después de regresar a la página anterior
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "#main .nasa-products-page-wrap"))
                        )
            
        except:
            continue


In [96]:
lista_nom_gen = sopa_producto.findAll("div", {"class" :"nasa-archive-product-warp"})

In [97]:
lista_nom_gen

[]

In [ ]:
<a href="https://www.maminess.com/producto/ropa-premama/vestidos-monos-premama/vestido-premama-punto-beige-frida/" title="Vestido Premamá Punto Beige Frida"><div class="main-img"><noscript><img width="450" height="600" src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-2-450x600.jpg" class="attachment-shop_catalog size-shop_catalog" alt="vestido-premama-punto-beige-frida" /></noscript><img width="450" height="600" src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-2-450x600.jpg" data-src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-2-450x600.jpg" class="attachment-shop_catalog size-shop_catalog ls-is-cached lazyloaded" alt="vestido-premama-punto-beige-frida"></div><div class="back-img back"><noscript><img width="450" height="600" src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-5-450x600.jpg" class="attachment-shop_catalog size-shop_catalog" alt="" /></noscript><img width="450" height="600" src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-5-450x600.jpg" data-src="https://www.maminess.com/wp-content/uploads/2022/12/vestido-premama-punto-beige-frida-5-450x600.jpg" class="attachment-shop_catalog size-shop_catalog ls-is-cached lazyloaded" alt=""></div> </a>

In [ ]:
<li class="cat-item cat-item-15 cat-item-ropa-premama root-item cat-parent li_accordion current-cat-parent active"><a href="javascript:void(0);" class="accordion" data-class_show="pe-7s-plus" data-class_hide="pe-7s-less"><span class="icon pe-7s-less"></span></a><a href="https://www.maminess.com/categoria/ropa-premama/" data-id="15" class="nasa-filter-by-cat">Ropa Premamá</a><ul class="children"><li class="cat-item cat-item-232 cat-item-vestidos-monos-premama current-cat active"><a href="https://www.maminess.com/categoria/ropa-premama/vestidos-monos-premama/" data-id="232" class="nasa-filter-by-cat nasa-active">Vestidos y monos Premamá</a></li><li class="cat-item cat-item-233 cat-item-tops-premama"><a href="https://www.maminess.com/categoria/ropa-premama/tops-premama/" data-id="233" class="nasa-filter-by-cat">Tops premamá</a></li><li class="cat-item cat-item-234 cat-item-pantalones-premama"><a href="https://www.maminess.com/categoria/ropa-premama/pantalones-premama/" data-id="234" class="nasa-filter-by-cat">Pantalones y Faldas</a></li><li class="cat-item cat-item-236 cat-item-banadores"><a href="https://www.maminess.com/categoria/ropa-premama/banadores/" data-id="236" class="nasa-filter-by-cat">Bañadores</a></li><li class="cat-item cat-item-237 cat-item-vestidos-fiesta-premama"><a href="https://www.maminess.com/categoria/ropa-premama/vestidos-fiesta-premama/" data-id="237" class="nasa-filter-by-cat">Vestidos de Fiesta</a></li><li class="cat-item cat-item-291 cat-item-pijamas-y-camisones"><a href="https://www.maminess.com/categoria/ropa-premama/pijamas-y-camisones/" data-id="291" class="nasa-filter-by-cat">Pijamas y Camisones</a></li><li class="cat-item cat-item-306 cat-item-promocion-hasta-60"><a href="https://www.maminess.com/categoria/ropa-premama/promocion-hasta-60/" data-id="306" class="nasa-filter-by-cat">Promo hasta -60%</a></li></ul></li>

In [174]:
lista_nombres = []
lista_precios = []
lista_nvaloraciones = []
lista_url_cat=["vestidos-monos-premama/", "ropa-premama/pantalones-premama/"]
for cat in lista_url_cat:
    for pag in range(1,4):
        try:
            driver = webdriver.Chrome()
            # Cargar la página
            url_mercados = f"https://www.maminess.com/categoria/ropa-premama/{cat}page/{pag}/"
            driver.get(url_mercados)
            driver.maximize_window()
            
            
            driver.execute_script("window.scrollBy(0, 1500);")
            sleep(1)
            contenido=driver.find_element(By.CSS_SELECTOR,"#main > div.nasa-archive-product-content > div.nasa-products-page-wrap.large-9.columns.right.has-sidebar > div.nasa-archive-product-warp").get_attribute("innerHTML")
            contenidoHTML = BeautifulSoup(contenido, "html.parser")


            #conseguir html producto
            lista_links_prod=contenidoHTML.find_all("div", {"class": "name"})
            lista_html_prod=[lil.find("a")["href"]for lil in lista_links_prod]
            
            #meto el htm de cada producto para sacar sus datos
            res = requests.get(lista_html_prod)
            sopa = BeautifulSoup(res.content, "html.parser")

            # Conseguir nombre
            lista_nom = sopa.findAll("h1", {"class": "product_title entry-title"})
            nombre = lista_nom[0].getText().lower().strip()
            print(nombre)
            lista_nombres.append(nombre)
            print(lista_nombres[-1])
            # Conseguir precio
            lista_prec = sopa.findAll("span", {"class": "woocommerce-Price-amount amount"})
            precio = lista_prec[1].getText().replace(",", ".").replace("€", "")
            lista_precios.append(precio)
            
            # Conseguir valoraciones
            lista_val = sopa.findAll("div", {"class": "nasa-tab-wrap text-center"})
            if lista_val:
                texto = lista_val[0].getText()
                numero = re.search(r'\((\d+)\)', texto)
                nvaloraciones = numero.group(1) if numero else "0"
            else:
                nvaloraciones = "No hay valoraciones"
            lista_nvaloraciones.append(nvaloraciones)

            
        




        except:
            continue

In [165]:
lista_nvaloraciones

[]

In [175]:
sopa

NameError: name 'sopa' is not defined

In [170]:
lista_nom = sopa.findAll("h1", {"class": "product_title entry-title"})
nombre = lista_nom[0].getText().lower().strip()
lista_nombres.append(nombre)

NameError: name 'sopa' is not defined

In [168]:
nombre

'peto vaquero premamá savanna'

In [171]:
lista_nombres

[]

In [141]:
len(contenidoHTML.find_all("div")[0].find_all("div"))

321

In [151]:
lista_links_prod=contenidoHTML.find_all("div", {"class": "name"})

In [156]:
lista_links_prod

[<div class="name"> <a href="https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaqueros-premama-sophia-gris/" title="Vaqueros Premamá Sophia Gris"> Vaqueros Premamá Sophia Gris </a></div>,
 <div class="name"> <a href="https://www.maminess.com/producto/ropa-premama/pantalones-premama/pantalon-deportivo-premama-tencel-negro/" title="Pantalón Deportivo Premamá Tencel Negro"> Pantalón Deportivo Premamá Tencel Negro </a></div>,
 <div class="name"> <a href="https://www.maminess.com/producto/ropa-premama/pantalones-premama/pantalon-deportivo-premama-sostenible-tancel-laurel/" title="Pantalón Deportivo Premamá Tencel Laurel"> Pantalón Deportivo Premamá Tencel Laurel </a></div>,
 <div class="name"> <a href="https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaqueros-premama-slim-alicia-2/" title="Vaqueros Premamá Slim Alicia"> Vaqueros Premamá Slim Alicia </a></div>,
 <div class="name"> <a href="https://www.maminess.com/producto/ropa-premama/vestidos-monos-premama

In [155]:
lista_links_prod[0].find("a")["href"]

'https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaqueros-premama-sophia-gris/'

In [159]:
lista_html_prod=[lil.find("a")["href"]for lil in lista_links_prod]

In [160]:
lista_html_prod

['https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaqueros-premama-sophia-gris/',
 'https://www.maminess.com/producto/ropa-premama/pantalones-premama/pantalon-deportivo-premama-tencel-negro/',
 'https://www.maminess.com/producto/ropa-premama/pantalones-premama/pantalon-deportivo-premama-sostenible-tancel-laurel/',
 'https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaqueros-premama-slim-alicia-2/',
 'https://www.maminess.com/producto/ropa-premama/vestidos-monos-premama/falda-premama-midi-flores/',
 'https://www.maminess.com/producto/ropa-premama/pantalones-premama/vaquero-premama-lola-slim-negro/']

In [132]:
lista_prec = sopa_producto.findAll("div", {"class" :"product-warp-item columns large-3 small-6 medium-6"})

#<span class="price"><del><span class="woocommerce-Price-amount amount"><bdi>44,90<span class="woocommerce-Price-currencySymbol">€</span></bdi></span></del> <ins><span class="woocommerce-Price-amount amount"><bdi>34,90<span class="woocommerce-Price-currencySymbol">€</span></bdi></span></ins></span>

In [133]:
lista_prec

[]

In [108]:
lista_prec
lista_precio=[prec.getText().strip().replace("€", "") for prec in lista_prec]
lista_precio

['44,90',
 '39,90',
 '',
 '',
 '',
 '69,90',
 '69,90',
 '44,90 19,90',
 '44,90 19,90',
 '26,90 18,90',
 '26,90 18,90',
 '29,90 24,90',
 '49,90 29,90',
 '49,90 29,90',
 '29,90 14,90',
 '29,90 14,90',
 '44,90 26,90',
 '44,90 26,90',
 '49,90 24,90',
 '49,90 24,90',
 '49,90 29,90',
 '49,90 29,90',
 '34,90',
 '34,90',
 '69,90 39,90',
 '69,90 39,90',
 '44,90 18,90',
 '44,90 18,90']